This notebook establishes a Validation and Testing Suite for the COPY INTO ingestion process (Chunk 1). 
It ensures that the dynamic table identifiers and target schemas meet enterprise standards before executing large-scale data movements.

####Why COPY INTO for Transaction Data?

We have selected the COPY INTO command for the Transactions dataset (Chunk 1) because it is the industry-standard choice for high-volume, batch-oriented CSV ingestion in Databricks.

 **Business & Industry Advantages
Idempotency & Cost Savings:**

- COPY INTO automatically keeps track of which files have already been loaded. From a business perspective, this prevents duplicate data and saves significant compute costs by avoiding the reprocessing of older files.

**Reliability in Batch Ingestion:** 

- In the retail and finance industries, transaction data is often delivered in large monthly or daily batches. COPY INTO is a robust, low-complexity tool that provides a simplified "SQL-first" experience for managing these loads.

**Built-in Auditability:**

- It supports the inclusion of file metadata (like _metadata.file_path) directly during the load, which is critical for meeting data governance and lineage requirements in regulated industries.

1. Core Ingestion Logic
This section isolates the logic used to build table paths and SQL commands, making the ingestion pipeline modular.

####Logic: 

The functions handle the dynamic construction of Unity Catalog identifiers and the generation of the COPY INTO SQL statement.

####Why this code: 
 
 Separating the string-building logic from the execution allows for Unit Testing. We can verify the SQL is correct before running it against a live cluster.

In [0]:
def get_table_identifier(catalog, schema, table="transactions_bronze"):
    """
    Logic: Dynamically constructs the 3-tier Unity Catalog table path.
    Why: Ensures the pipeline can target different environments (Dev/Prod) without hardcoding.
    """
    if not catalog or not schema:
        return f"test_temp_{table}" # Fallback for local/CI testing
    return f"`{catalog}`.`{schema}`.`{table}`"

def get_ingestion_sql(table_name, source_path):
    """
    Logic: Generates the specialized COPY INTO command.
    Why: Centralizes the command structure for easier maintenance and schema evolution.
    """
    return f"COPY INTO {table_name} FROM '{source_path}' FILEFORMAT = CSV"

##2. Unit and Integration Test Suite

This block validates the logic and the environment using the unittest framework and Spark testing utilities.

####Logic:

- **Path Generation Test:** Validates that the table identifier correctly follows the `catalog`.`schema`.`table` format.

- **Schema Audit Test:** Verifies that the required audit columns (load_dt and source_file) are present in the target table schema.

####Why this code:

These tests serve as a "Pre-Flight Check." By validating the schema and paths, we ensure the pipeline won't fail midway through a multi-million row transaction load.

In [0]:
import unittest
import io
from unittest import TextTestRunner
from pyspark.testing.utils import assertSchemaEqual
from pyspark.sql.types import StructType, StructField, StringType, TimestampType

class IngestionTest(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        cls.spark = spark
        cls.test_table = "temp_test_ingest"

    def test_dynamic_path_generation(self):
        """Unit Test: Ensure table identifier logic is correct and escaped"""
        actual = get_table_identifier("dev", "raw", "orders")
        self.assertEqual(actual, "`dev`.`raw`.`orders`")

    def test_schema_audit_columns(self):
        """Integration Test: Validate existence of mandatory audit columns"""
        # Setup: Create a local mock table with audit columns
        self.spark.sql(f"CREATE TABLE IF NOT EXISTS {self.test_table} (load_dt TIMESTAMP, source_file STRING)")
        
        expected_schema = StructType([
            StructField("load_dt", TimestampType(), True),
            StructField("source_file", StringType(), True)
        ])
        
        actual_schema = self.spark.table(self.test_table).schema
        # Logic: Confirm the actual table matches the required enterprise audit schema
        assertSchemaEqual(actual_schema, expected_schema)

# Load the tests into the execution suite
suite = unittest.TestLoader().loadTestsFromTestCase(IngestionTest)

##3.Reporting and Quality Verdict

The final section executes the tests and generates a formatted Ingestion Pipeline Quality Report.

####Logic: 

It captures the test output into a string buffer to present a clean summary of successes and failures. 

####Why this code:

This provides a clear "Success" signal to the data engineering team. If the report shows failures, the notebook includes instructions to use %debug to inspect the logic immediately.

In [0]:
# 1. Capture report in a stream
stream = io.StringIO()
runner = TextTestRunner(stream=stream, verbosity=2)
result = runner.run(suite)

# 2. Format the Final Report
print("●●● INGESTION PIPELINE QUALITY REPORT ●●●")
print("-" * 45)
print(stream.getvalue())
print("-" * 45)
print(f"TOTAL TESTS: {result.testsRun}")
print(f"SUCCESSES: {result.testsRun - len(result.failures) - len(result.errors)}")
print(f"FAILURES/ERRORS: {len(result.failures) + len(result.errors)}")
print("-" * 45)

if not result.wasSuccessful():
    print("\n[INSTRUCTION] To debug the ERROR above, run '%debug' in the next cell.")

**Report Summary**
- Total Tests Run: 2

- Successes: 2

- Status: ✅ OK